# Домашнее задание №8. Бустинг и кластеризация.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

## Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [ ]:
link = 'https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2024/homeworks/hw08_boosting_clustering/ds_salaries.csv'

In [ ]:
df = pd.read_csv(link)
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


### Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [ ]:
df = df.drop('salary', axis=1)

In [ ]:
X, y = df.drop('salary_in_usd', axis=1), df['salary_in_usd']

In [ ]:
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=42)

In [ ]:
X_train

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
2238,2022,SE,FT,Data Engineer,EUR,ES,0,ES,M
485,2023,MI,FT,Research Scientist,USD,US,100,US,M
2177,2022,SE,FT,Data Analyst,USD,US,0,US,M
3305,2022,SE,FT,Data Engineer,USD,US,100,US,M
1769,2023,SE,FT,Data Engineer,USD,US,100,US,M
...,...,...,...,...,...,...,...,...,...
1130,2023,SE,FT,Machine Learning Engineer,USD,US,100,US,M
1294,2023,SE,FT,Data Analyst,USD,US,0,US,M
860,2023,EX,FT,Data Engineer,USD,US,100,US,M
3507,2021,MI,FT,Research Scientist,EUR,FR,50,FR,M


### Задание 2 (0.5 балла) Линейная модель

*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

In [ ]:
one_hot_encoded = enc.fit_transform(X_train[categorical_features])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(categorical_features))

In [ ]:
X_train['temp'] = range(X_train.shape[0])

In [ ]:
X_train_encoded = pd.merge(X_train, one_hot_df, left_on='temp', right_index=True)
X_train_encoded = X_train_encoded.drop(categorical_features + ['temp'], axis=1)

In [ ]:
X_train = X_train.drop('temp', axis=1)

In [ ]:
one_hot_encoded = enc.transform(X_val[categorical_features])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(categorical_features))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2, 3, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
X_val['temp'] = range(X_val.shape[0])

In [ ]:
X_val_encoded = pd.merge(X_val, one_hot_df, left_on='temp', right_index=True)
X_val_encoded = X_val_encoded.drop(categorical_features + ['temp'], axis=1)

In [ ]:
X_val = X_val.drop('temp', axis=1)

In [ ]:
one_hot_encoded = enc.transform(X_test[categorical_features])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(categorical_features))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
X_test['temp'] = range(X_test.shape[0])

In [ ]:
X_test_encoded = pd.merge(X_test, one_hot_df, left_on='temp', right_index=True)
X_test_encoded = X_test_encoded.drop(categorical_features + ['temp'], axis=1)

In [ ]:
X_test = X_test.drop('temp', axis=1)

In [ ]:
lin_reg = LinearRegression()

In [ ]:
lin_reg = lin_reg.fit(X_train_encoded, y_train)

In [ ]:
y_val_pred = lin_reg.predict(X_val_encoded)
y_test_pred = lin_reg.predict(X_test_encoded)

In [ ]:
print('MAPE on Validation:', mean_absolute_percentage_error(y_val_pred, y_val))
print('RMSE on Validation:', mean_squared_error(y_val_pred, y_val) ** 0.5)
print('\nMAPE on Test:', mean_absolute_percentage_error(y_test_pred, y_test))
print('RMSE on Test:', mean_squared_error(y_test_pred, y_test) ** 0.5)

MAPE on Validation: 0.551802382221792
RMSE on Validation: 45854219321146.02

MAPE on Test: 0.32827032661916294
RMSE on Test: 1976742065465.0422


### Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
import optuna
from xgboost.sklearn import XGBRegressor

In [ ]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }

    model = XGBRegressor(**param)
    model.fit(X_train_encoded, y_train)
    y_pred = model.predict(X_val_encoded)
    return mean_squared_error(y_pred, y_val)

In [ ]:
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=200)

[I 2024-06-08 16:35:23,191] A new study created in memory with name: regression
[I 2024-06-08 16:35:24,291] Trial 0 finished with value: 2337899218.3700037 and parameters: {'max_depth': 4, 'learning_rate': 0.8696786436937256, 'n_estimators': 170, 'min_child_weight': 6, 'gamma': 0.05244629602289512, 'subsample': 0.9423944087867498, 'colsample_bytree': 0.9907940667983337, 'reg_alpha': 0.16061900575252558, 'reg_lambda': 0.2913959098186366, 'random_state': 988}. Best is trial 0 with value: 2337899218.3700037.
[I 2024-06-08 16:35:26,477] Trial 1 finished with value: 2797635287.5305066 and parameters: {'max_depth': 6, 'learning_rate': 0.5958265531404647, 'n_estimators': 469, 'min_child_weight': 7, 'gamma': 0.5194257371786687, 'subsample': 0.24272917648863923, 'colsample_bytree': 0.6382877686834338, 'reg_alpha': 0.35263506693335395, 'reg_lambda': 0.9666325552264446, 'random_state': 664}. Best is trial 0 with value: 2337899218.3700037.
[I 2024-06-08 16:35:28,053] Trial 2 finished with value: 2

In [ ]:
print('Best MSE on Validation', study.best_value)

Best MSE on Validation 2020973233.9059641


In [ ]:
xgboost_reg = XGBRegressor(**study.best_params)

In [ ]:
xgboost_reg = xgboost_reg.fit(X_train_encoded, y_train)

In [ ]:
y_pred = xgboost_reg.predict(X_test_encoded)

In [ ]:
print('MAPE on Test:', mean_absolute_percentage_error(y_pred, y_test))
print('RMSE on Test:', mean_squared_error(y_pred, y_test) ** 0.5)

MAPE on Test: 0.29924642148198927
RMSE on Test: 50718.48055617091


Скорость обучения (оптимального подбора гиперпараметров) занимает около 5 минут, скорость предсказания почти моментальная, чуть-чуть медленнее, чем в обычной лин. регрессии (по-крайней мере я не заметил).

Само же обучение модели, после подбора гиперпараметров, является чуть более заметным, в сравнении с той же линейной моделью. Обучается порядка 1-2 секунд.

P.S. вдохновился идеей такого подбора гиперпараметров [отсюда](https://practicaldatascience.co.uk/machine-learning/how-to-tune-an-xgbregressor-model-with-optuna) :)

### Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.7 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
def objective(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }
    model = CatBoostRegressor(**params, silent=True)
    model.fit(X_train_encoded, y_train)
    y_pred = model.predict(X_val_encoded)
    return mean_squared_error(y_pred, y_val)

In [ ]:
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=200)

[I 2024-06-08 16:53:40,186] A new study created in memory with name: regression
[I 2024-06-08 16:53:50,937] Trial 0 finished with value: 2108834617.328911 and parameters: {'learning_rate': 0.005748589230872894, 'depth': 10, 'subsample': 0.34551261123867516, 'colsample_bylevel': 0.10737970897355106, 'min_data_in_leaf': 33}. Best is trial 0 with value: 2108834617.328911.
[I 2024-06-08 16:53:55,146] Trial 1 finished with value: 2041818914.0988815 and parameters: {'learning_rate': 0.024118027618617963, 'depth': 9, 'subsample': 0.14175827225507664, 'colsample_bylevel': 0.2921044282931491, 'min_data_in_leaf': 91}. Best is trial 1 with value: 2041818914.0988815.
[I 2024-06-08 16:53:55,771] Trial 2 finished with value: 2099010162.6183598 and parameters: {'learning_rate': 0.021734822627858735, 'depth': 2, 'subsample': 0.35088225010864466, 'colsample_bylevel': 0.9313715443997924, 'min_data_in_leaf': 91}. Best is trial 1 with value: 2041818914.0988815.
[I 2024-06-08 16:53:56,620] Trial 3 finished

In [ ]:
print('Best MSE on Validation', study.best_value)

Best MSE on Validation 2005260290.9176452


In [ ]:
catboost_reg = CatBoostRegressor(**study.best_params, silent=True)

In [ ]:
catboost_reg = catboost_reg.fit(X_train_encoded, y_train)

In [ ]:
y_pred = catboost_reg.predict(X_test_encoded)

In [ ]:
print('MAPE on Test:', mean_absolute_percentage_error(y_pred, y_test))
print('RMSE on Test:', mean_squared_error(y_pred, y_test) ** 0.5)

MAPE on Test: 0.33071333673237324
RMSE on Test: 50184.417090324016


Catboost учится значительно медленнее, чем XGBoost (примерно в 6-8 раз дольше), а предиктит с такой же скоростью.

Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [ ]:
from catboost import Pool

In [ ]:
train_data = Pool(data=X_train, label=y_train, cat_features=categorical_features)
val_data = Pool(data=X_val, label=y_val, cat_features=categorical_features)
test_data = Pool(data=X_test, label=y_test, cat_features=categorical_features)

In [ ]:
def objective(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }
    model = CatBoostRegressor(**params, silent=True)
    model.fit(train_data)
    y_pred = model.predict(val_data)
    return mean_squared_error(y_pred, y_val)

In [ ]:
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=200)

[I 2024-06-08 17:43:20,143] A new study created in memory with name: regression
[I 2024-06-08 17:43:22,086] Trial 0 finished with value: 2706719550.856371 and parameters: {'learning_rate': 0.0015807441421211933, 'depth': 1, 'subsample': 0.8260222457483726, 'colsample_bylevel': 0.2379339068779528, 'min_data_in_leaf': 94}. Best is trial 0 with value: 2706719550.856371.
[I 2024-06-08 17:43:29,951] Trial 1 finished with value: 2061931095.733998 and parameters: {'learning_rate': 0.0054403049205478254, 'depth': 6, 'subsample': 0.5081501256197366, 'colsample_bylevel': 0.5876036858877162, 'min_data_in_leaf': 86}. Best is trial 1 with value: 2061931095.733998.
[I 2024-06-08 17:43:39,412] Trial 2 finished with value: 2111431979.4955776 and parameters: {'learning_rate': 0.002536284293702799, 'depth': 8, 'subsample': 0.6605164368587562, 'colsample_bylevel': 0.7358487368326524, 'min_data_in_leaf': 13}. Best is trial 1 with value: 2061931095.733998.
[I 2024-06-08 17:43:41,704] Trial 3 finished with 

In [ ]:
print('Best MSE on Validation', study.best_value)

Best MSE on Validation 2046786642.2282145


In [ ]:
catboost_reg = CatBoostRegressor(**study.best_params, silent=True)

In [ ]:
catboost_reg = catboost_reg.fit(train_data)

In [ ]:
y_pred = catboost_reg.predict(test_data)

In [ ]:
print('MAPE on Test:', mean_absolute_percentage_error(y_pred, y_test))
print('RMSE on Test:', mean_squared_error(y_pred, y_test) ** 0.5)

MAPE on Test: 0.2970282861818719
RMSE on Test: 50594.025667795504


С помощью Pool Catboost обучается значительно быстрее (чуть быстрее, чем XGBoost). При этом в качестве мы практически не потеряли (спишу эту потерю на то, что гиперпараметры каждый раз выбираются рандомно и в прошлый раз нам повезло чуть больше).

### Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.

In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
def objective(trial):
    params = {
        "n_estimators": 1000,
        "verbosity": -1,
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = LGBMRegressor(**params)
    model.fit(X_train_encoded, y_train)
    y_pred = model.predict(X_val_encoded)
    return mean_squared_error(y_pred, y_val)

In [ ]:
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=200)

[I 2024-06-08 18:01:07,940] A new study created in memory with name: regression
[I 2024-06-08 18:01:09,428] Trial 0 finished with value: 2313463029.382822 and parameters: {'learning_rate': 0.05181600784545532, 'num_leaves': 500, 'subsample': 0.7258439951556614, 'colsample_bytree': 0.852926886085999, 'min_data_in_leaf': 13}. Best is trial 0 with value: 2313463029.382822.
[I 2024-06-08 18:01:10,552] Trial 1 finished with value: 2132259369.231388 and parameters: {'learning_rate': 0.006030515163026436, 'num_leaves': 502, 'subsample': 0.7031706222625088, 'colsample_bytree': 0.8343270795083182, 'min_data_in_leaf': 19}. Best is trial 1 with value: 2132259369.231388.
[I 2024-06-08 18:01:10,973] Trial 2 finished with value: 2761675455.554679 and parameters: {'learning_rate': 0.0010798315877628619, 'num_leaves': 285, 'subsample': 0.9389494414687402, 'colsample_bytree': 0.18239128340216038, 'min_data_in_leaf': 44}. Best is trial 1 with value: 2132259369.231388.
[I 2024-06-08 18:01:11,280] Trial 3

In [ ]:
print('Best MSE on Validation', study.best_value)

Best MSE on Validation 2018889981.3767686


In [ ]:
params = study.best_params
params['verbosity'] = -1
params['bagging_freq'] = 1
params['n_estimators'] = 1000

In [ ]:
lightgbm_reg = LGBMRegressor(**params)

In [ ]:
lightgbm_reg.fit(X_train_encoded, y_train)

LGBMRegressor(bagging_freq=1, colsample_bytree=0.2698661544550345,
              learning_rate=0.006833977455072198, min_data_in_leaf=1,
              n_estimators=1000, num_leaves=919, subsample=0.9547278726573168,
              verbosity=-1)

In [ ]:
y_pred = lightgbm_reg.predict(X_test_encoded)

In [ ]:
print('MAPE on Test:', mean_absolute_percentage_error(y_pred, y_test))
print('RMSE on Test:', mean_squared_error(y_pred, y_test) ** 0.5)

MAPE on Test: 0.2770958880889578
RMSE on Test: 51081.165052918725


Качество также на уровне, показатели ошибок схожи с другими двумя рассмотренными бустами. По времени обучения, занимает побольше, чем XGBoost и Catboost с Pool.


### Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

In [ ]:
xgboost_params = {'max_depth': 1, 'learning_rate': 0.1514582057908884, 'n_estimators': 817, 'min_child_weight': 7, 'gamma': 0.1012889421408612, 'subsample': 0.8920316112387824, 'colsample_bytree': 0.4406936158248896, 'reg_alpha': 0.7488939245442084, 'reg_lambda': 0.33225764793617774, 'random_state': 458}
catboost_no_pool_params = {'n_estimators': 1000, 'learning_rate': 0.05134107603295446, 'depth': 10, 'subsample': 0.3308956136177017, 'colsample_bylevel': 0.0734889015571684, 'min_data_in_leaf': 90}
catboost_pool_params = {'n_estimators': 1000, 'learning_rate': 0.019786685491402396, 'depth': 2, 'subsample': 0.42103654805977664, 'colsample_bylevel': 0.1888054711001659, 'min_data_in_leaf': 35}
lightgbm_params = {'n_estimators': 1000, 'verbosity': -1,  'bagging_freq': 1, 'learning_rate': 0.006833977455072198, 'num_leaves': 919, 'subsample': 0.9547278726573168, 'colsample_bytree': 0.2698661544550345, 'min_data_in_leaf': 1}

In [ ]:
xgboost_reg = XGBRegressor(**xgboost_params)
catboost_no_pool_reg = CatBoostRegressor(**catboost_no_pool_params, silent=True)
catboost_pool_reg = CatBoostRegressor(**catboost_pool_params, silent=True)
lightgbm_reg = LGBMRegressor(**lightgbm_params)

**Время обучения**

In [ ]:
%%time
xgboost_reg = xgboost_reg.fit(X_train_encoded, y_train)

CPU times: user 2.78 s, sys: 20.1 ms, total: 2.8 s
Wall time: 1.47 s


In [ ]:
%%time
catboost_no_pool_reg = catboost_no_pool_reg.fit(X_train_encoded, y_train)

CPU times: user 11.6 s, sys: 800 ms, total: 12.4 s
Wall time: 8.18 s


In [ ]:
%%time
catboost_pool_reg = catboost_pool_reg.fit(train_data)

CPU times: user 1.06 s, sys: 177 ms, total: 1.24 s
Wall time: 677 ms


In [ ]:
%%time
lightgbm_reg = lightgbm_reg.fit(X_train_encoded, y_train)

CPU times: user 3.64 s, sys: 244 ms, total: 3.89 s
Wall time: 3.87 s


Быстрее всех обучается Catboost с Pool, медленнее всех также Catboost, но без Pool.

**Время предикта**

In [ ]:
%%time
y_pred_xgboost = xgboost_reg.predict(X_test_encoded)

CPU times: user 51.4 ms, sys: 0 ns, total: 51.4 ms
Wall time: 56 ms


In [ ]:
%%time
y_pred_catboost_no_pool = catboost_no_pool_reg.predict(X_test_encoded)

CPU times: user 20.8 ms, sys: 1.06 ms, total: 21.9 ms
Wall time: 26.5 ms


In [ ]:
%%time
y_pred_catboost_pool = catboost_pool_reg.predict(test_data)

CPU times: user 6.6 ms, sys: 1.97 ms, total: 8.57 ms
Wall time: 11.2 ms


In [ ]:
%%time
y_pred_lightgbm = lightgbm_reg.predict(X_test_encoded)

CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 132 ms


Быстрее всех предсказывает также Catboost с Pool, но при этом дольше всех предсказывает LightGBM.

**Качество**

In [ ]:
print('MAPE on Test:', mean_absolute_percentage_error(y_pred_xgboost, y_test))
print('RMSE on Test:', mean_squared_error(y_pred_xgboost, y_test) ** 0.5)

MAPE on Test: 0.29924642148198927
RMSE on Test: 50718.48055617091


In [ ]:
print('MAPE on Test:', mean_absolute_percentage_error(y_pred_catboost_no_pool, y_test))
print('RMSE on Test:', mean_squared_error(y_pred_catboost_no_pool, y_test) ** 0.5)

MAPE on Test: 0.33071333673237324
RMSE on Test: 50184.417090324016


In [ ]:
print('MAPE on Test:', mean_absolute_percentage_error(y_pred_catboost_pool, y_test))
print('RMSE on Test:', mean_squared_error(y_pred_catboost_pool, y_test) ** 0.5)

MAPE on Test: 0.2970282861818719
RMSE on Test: 50594.025667795504


In [ ]:
print('MAPE on Test:', mean_absolute_percentage_error(y_pred_lightgbm, y_test))
print('RMSE on Test:', mean_squared_error(y_pred_lightgbm, y_test) ** 0.5)

MAPE on Test: 0.2770958880889578
RMSE on Test: 51081.165052918725


В плане качества по RMSE лучше всех себя показал Catboost без Pool, а по MAPE - LightGBM.

**Гиперпараметры**

В XGBoost получился самый большой `learning_rate` (0.15) и достаточно большой `subsample (0.89)`, однако в LightGBM этот показатель еще выше - 0.95.

`min_data_in_leaf` в Catboost без Pool оказался чрезмерно большим (90) как и глубина деревьев (аж 10), в то время как в LightGBM `min_data_in_leaf` оказался очень маленьким (1).